Load libraries

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

load dataset

In [37]:
retail_raw = pd.read_csv("../data/retail_raw.csv")
retail_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


get inital sumary

In [38]:
before_summary = pd.DataFrame({
    "dtype": retail_raw.dtypes,
    "missing_values": retail_raw.isnull().sum(),
    "non_missing": retail_raw.notnull().sum(),
    "unique_values": retail_raw.nunique()
})
before_summary

,dtype,missing_values,non_missing,unique_values
InvoiceNo,object,0,541909,25900
StockCode,object,0,541909,4070
Description,object,1454,540455,4223
Quantity,int64,0,541909,722
InvoiceDate,object,0,541909,23260
UnitPrice,float64,0,541909,1630
CustomerID,int64,0,541909,4372
Country,object,0,541909,38


look at shape:

In [39]:
retail_raw.shape

(541909, 8)

Look at the describe:

In [40]:
retail_raw.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,541909.000000
mean,9.552250,4.611114,15287.518434
std,218.081158,96.759853,1484.746041
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,14367.000000
50%,3.000000,2.080000,15287.000000
75%,10.000000,4.130000,16255.000000
max,80995.000000,38970.000000,18287.000000


Put findings here.....

set invoice date as type date

In [41]:
retail_raw['InvoiceDate'] = pd.to_datetime(retail_raw['InvoiceDate'])
retail_raw.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID              int64
Country                object
dtype: object

check counts by day:

In [42]:
retail_raw['InvoiceDate'] = pd.to_datetime(retail_raw['InvoiceDate'])
weekday_counts = retail_raw['InvoiceDate'].dt.day_name().value_counts()
weekday_counts

InvoiceDate
Thursday     103857
Tuesday      101808
Monday        95111
Wednesday     94565
Friday        82193
Sunday        64375
Name: count, dtype: int64

talk about no saturday

Look at duplciates:

In [43]:
retail_raw.duplicated().sum()

np.int64(5268)

there are 5268 duolicates, drop them:

In [44]:
retail_raw.drop_duplicates(inplace=True)

retail_raw.duplicated().sum()

np.int64(0)

remove non positive prices and save as a new dataframe

In [45]:
retail = retail_raw[retail_raw['UnitPrice'] > 0]
retail.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,534129.000000,534129,534129.000000,534129.000000
mean,9.916818,2011-07-04 12:02:01.631403776,4.695864,15282.689715
min,-80995.000000,2010-12-01 08:26:00,0.001000,12346.000000
25%,1.000000,2011-03-28 11:36:00,1.250000,14344.000000
50%,3.000000,2011-07-19 15:55:00,2.100000,15287.000000
75%,10.000000,2011-10-18 17:10:00,4.130000,16248.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,216.452113,NaN,95.079189,1486.141831


look at negative quantaties

In [46]:
retail[retail['Quantity'] < 0].head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
239,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548,United Kingdom
240,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
241,C536391,22553,PLASTERS IN TIN SKULLS,-24,2010-12-01 10:24:00,1.65,17548,United Kingdom
939,C536506,22960,JAM MAKING SET WITH JARS,-6,2010-12-01 12:38:00,4.25,17897,United Kingdom


explain the C part and include link